In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 100)
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
CSV_DATA = Path().cwd().parent.parent / "data/1_transformed/"

In [ ]:
def plot_state_column(df:pd.DataFrame, column:str, color_by:str='segment_id',figsize=(15,3),
                       ylims:list[float]=None, units:str='none', show_mean:bool=True) -> None:
    _ = plt.figure(figsize=figsize)
    _ = plt.hlines(y=df[column].mean(), xmin=1, xmax=df['elapsed_time'].max(), colors='red', linestyles='dashed')
    _ = sns.lineplot(df, x='elapsed_time', y=column, hue=color_by, palette='hls', legend=False)
    _ = plt.grid()
    _ = plt.title(f'{column.upper()} Segments', fontsize=18)
    _ = plt.xlabel('Elapsed Time (seconds)', fontsize=14)
    _ = plt.ylabel(f'{column.upper()} ({units})', fontsize=14)
    
    if ylims is not None:
        _ = plt.ylim(ylims)

# 1. Load Ride Data

In [ ]:
df = pd.read_csv(CSV_DATA / "0x66780269.csv")

In [ ]:
# How many miles was this ride?
df['delta_dist_ft'].sum() / 5280

In [ ]:
df.columns

In [ ]:
cols_of_interest = ['time','segment_id','elapsed_time','delta_time','delta_dist_ft','speed','grade_saturated','filt_speed','filt_grade_saturated','inst_power',]
df = df.loc[:,cols_of_interest].copy()
df.head()

In [ ]:
plot_state_column(df, 'filt_speed')

In [ ]:
plot_state_column(df, 'filt_grade_saturated')

In [ ]:
plot_state_column(df, 'inst_power')

In [ ]:
def calculate_power_curve(df: pd.DataFrame):
    # Create a set of rolling windows to calculate a MAX over avg(inst_powers[within_window])
    rolling_windows = [4, 5, 10, 20, 30, 60, # seconds
                        2*60, 3*60, 4*60, 5*60, 6*60, 10*60, 20*60, 30*60, 40*60, # minutes
                        60*60, 2*60*60, 3*60*60, 4*60*60] # hours
    rwindow_labels = ['4s', '5s', '10s', '20s', '30s', '1m', # seconds
                        '2m', '3m', '4m', '5m', '6m', '10m', '20m', '30m', '40m', # minutes
                        '1h', '2h', '3h', '4h'] # hours
    label_map = {seconds:label for seconds,label in zip(rolling_windows,rwindow_labels)}

    # Initialize a list to store the peak powers per window
    window_peak_powers = []

    for rwindow in rolling_windows:
        # We should not calculate the peak power for a window that is longer than this value since it is ill defined
        rolling_avg_inst_power = df[['inst_power']].rolling(rwindow, min_periods=rwindow).mean().dropna()
        if rolling_avg_inst_power.shape[0]==0: # all values were np.nan, hence the window is too large for the ride data
            peak_power = np.nan
        else:
            peak_power = max(rolling_avg_inst_power.values)[0]
        window_peak_powers.append({'time_window':label_map[rwindow], 'window_length_seconds':rwindow, 'peak_avg_power':peak_power})
    
    return pd.DataFrame(window_peak_powers)

In [ ]:
df_pwr = calculate_power_curve(df=df)
df_pwr

In [ ]:
for _, row in df_pwr[['time_window','peak_avg_power']].iterrows():
    print(row['time_window'], row['peak_avg_power'])

In [ ]:
def plot_ride_power_curve(df:pd.DataFrame, figsize:tuple=(12,4), logscale_base:int=10,
                          title:str='Best Ride Efforts Power Curve'):
    df_pwr = calculate_power_curve(df=df)
    x_col = 'window_length_seconds'

    fig, ax = plt.subplots(1,1, figsize=figsize)
    _ = sns.lineplot(df_pwr, x=x_col, y='peak_avg_power', ax=ax)
    _ = sns.scatterplot(df_pwr, x=x_col, y='peak_avg_power', ax=ax)
    
    def forward(x):
        return np.log10(x) / np.log10(logscale_base)

    def inverse(x):
        return logscale_base**x
    
    if logscale_base==10:
        _ = ax.set_xscale('log')
    else:
        _ = ax.set_xscale('function', functions=(forward,inverse))
    _ = plt.xticks(ticks=df_pwr['window_length_seconds'].values, labels=df_pwr['time_window'].values, rotation=45)
    _ = plt.grid()
    _ = ax.set_axisbelow(True)
    _ = plt.xlabel('Effort Time Duration', fontsize=14)
    _ = plt.ylabel('Maximum Effort Power (W)', fontsize=14)
    _ = plt.title(title, fontsize=18)

In [ ]:
plot_ride_power_curve(df)

# 2. Analyzing a Ride with Heart Rate

In [ ]:
df2 = pd.read_csv(CSV_DATA / "0x680b85ff.csv") 
print(f'the ride is {df2["delta_dist_ft"].sum() * 1/5280.0} miles')

In [ ]:
cols_of_interest = ['time','segment_id','elapsed_time','delta_time','delta_dist_ft','speed','grade_saturated','filt_speed','filt_grade_saturated','inst_power',
                    'heart_rate_bpm']
df2 = df2.loc[:,cols_of_interest].copy()
df2.head()

In [ ]:
plot_ride_power_curve(df2)

In [ ]:
plot_state_column(df2, 'filt_speed')

In [ ]:
plot_state_column(df2, 'heart_rate_bpm')

In [ ]:
longest_ride = '0x66fbe94a.csv'
hardest_otet_ride = '0x670e5fca.csv'

In [ ]:
df3 = pd.read_csv(CSV_DATA / longest_ride) 
print(f'the "longest_ride" is {df3["delta_dist_ft"].sum() * 1/5280.0} miles')
df4 = pd.read_csv(CSV_DATA / hardest_otet_ride) 
print(f'the "hardest_otet_ride" is {df4["delta_dist_ft"].sum() * 1/5280.0} miles')

In [ ]:
cols_of_interest = ['time','segment_id','elapsed_time','delta_time','delta_dist_ft','speed','grade_saturated','filt_speed','filt_grade_saturated','inst_power',
                    'heart_rate_bpm']
df3 = df3.loc[:,cols_of_interest].copy()
df4 = df4.loc[:,cols_of_interest].copy()

In [ ]:
plot_ride_power_curve(df3)

In [ ]:
plot_ride_power_curve(df4)

In [ ]:
plot_state_column(df3, 'heart_rate_bpm')

In [ ]:
plot_state_column(df4, 'heart_rate_bpm')

# Inspecting Rides with Erroneous Power Curves (too high)

In [ ]:
sample1 = '0x6647e479.csv' # May 17, 2024
sample2 = '0x647fc3f0.csv' # June 6, 2023

In [ ]:
df5 = pd.read_csv(CSV_DATA / sample1) 
df6 = pd.read_csv(CSV_DATA / sample2) 

In [ ]:
plot_state_column(df5, 'speed')

In [ ]:
plot_state_column(df5, 'filt_speed')

In [ ]:
plot_state_column(df5, 'inst_power')

In [ ]:
###############################################

In [ ]:
plot_state_column(df6, 'speed')

In [ ]:
plot_state_column(df6, 'filt_speed')

In [ ]:
plot_state_column(df6, 'inst_power')